In [1]:
import pvdeg
import pygmt

import xarray as xr
import numpy as np

from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

AttributeError: `np.Inf` was removed in the NumPy 2.0 release. Use `np.inf` instead.

In [ ]:
visual_demo_ds = xr.open_dataset("demos/freeze-thaw-us.nc")

In [ ]:
# fig, ax = pvdeg.geospatial.plot_sparse_analysis(visual_demo_ds, data_var = 'cycles', method='linear', res=400j)

In [ ]:
lon_min, lat_min, lon_max, lat_max = [
    visual_demo_ds.longitude.min(),
    visual_demo_ds.latitude.min(),
    visual_demo_ds.longitude.max(),
    visual_demo_ds.latitude.max(),
]

In [ ]:
relief_gmt = pygmt.datasets.load_earth_relief(resolution="10m")

relief_ds = relief_gmt.to_dataset().rename(
    {
        "lat": "latitude",
        "lon": "longitude",
    }
)

subset = relief_ds.sel(
    latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max)
)

In [ ]:
results_holder = pvdeg.scenario.GeospatialScenario()

results_holder.results = subset

results_holder.plot_world(data_variable="z")

In [ ]:
# MUST REVERSE ROW ORDERS
reversed = subset.z.values[::-1, :]

# Step 2: Generate Hillshading and Color Overlay
ls = LightSource(azdeg=315, altdeg=45)
hillshade = ls.hillshade(reversed, vert_exag=2.5, dx=1, dy=1)

# Blend the hillshade with a colormap for elevation
colored_hillshade = ls.shade(
    reversed, cmap=plt.cm.terrain, vert_exag=2.5, blend_mode="soft"
)

# Step 3: Plot the Hillshaded Relief with Cartopy
fig = plt.figure(figsize=(12, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Overlay the blended hillshade with color
ax.imshow(
    colored_hillshade,
    extent=[lon_min, lon_max, lat_min, lat_max],
    transform=ccrs.PlateCarree(),
    origin="upper",
)

# Add map features (coastlines, borders, etc.)
ax.add_feature(cfeature.COASTLINE, edgecolor="black", linewidth=0.5)
ax.add_feature(cfeature.BORDERS, edgecolor="black", linewidth=0.3)
ax.add_feature(cfeature.LAKES, edgecolor="blue", facecolor="none")

# Optional: Add gridlines for globe-like appearance
gl = ax.gridlines(
    draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--"
)
gl.top_labels = False
gl.right_labels = False

# Step 4: Add Title and Display
plt.title("3D Terrain with Hillshading", fontsize=16)
plt.show()

In [ ]:
# Generate the fake elevation map (reversed and hillshaded)
elevation_reversed = subset.z.values[::-1, :]
ls = LightSource(azdeg=315, altdeg=45)
hillshade = ls.hillshade(elevation_reversed, vert_exag=2.5, dx=1, dy=1)

# Step 3: Plot the Hillshaded Relief with Cartopy
fig = plt.figure(figsize=(12, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Plot the sparse analysis
fig, ax = pvdeg.geospatial.plot_sparse_analysis(
    visual_demo_ds, data_var="cycles", method="linear", res=400j, ax=ax
)

# Overlay the elevation map (hillshade) on the existing axis
ax.imshow(
    hillshade,
    extent=[lon_min, lon_max, lat_min, lat_max],  # Ensure extents match
    transform=ccrs.PlateCarree(),  # Same projection as the base plot
    origin="upper",
    cmap="gray",  # Grayscale hillshading
    alpha=0.1,  # Adjust transparency for better visibility
)

# Optional: Add additional map features or enhancements
ax.add_feature(cfeature.COASTLINE, edgecolor="black", linewidth=0.5)
ax.add_feature(cfeature.BORDERS, edgecolor="black", linewidth=0.3)
ax.add_feature(cfeature.LAKES, edgecolor="blue", facecolor="none")

# Optional: Add a title or modify other properties of the plot
plt.title("Sparse Analysis with Elevation Overlay", fontsize=16)

# Display the plot
plt.show()

In [ ]:
from matplotlib.colors import Normalize


# Step 1: Generate the Normal Map
def generate_normal_map(heightmap):
    # Compute gradients in x and y directions
    grad_x, grad_y = np.gradient(heightmap)

    # Compute z gradient (constant, for scaling normals)
    grad_z = np.ones_like(heightmap)

    # Normalize the gradients to get surface normals
    length = np.sqrt(grad_x**2 + grad_y**2 + grad_z**2)
    nx = grad_x / length
    ny = grad_y / length
    nz = grad_z / length

    # Map normals to RGB values (scaled from 0 to 1)
    normal_map = np.stack(
        (
            (nx + 1) / 2,  # Map x normal to red
            (ny + 1) / 2,  # Map y normal to green
            (nz + 1) / 2,  # Map z normal to blue
        ),
        axis=-1,
    )

    return normal_map


# Generate the fake elevation map (reversed and hillshaded)
elevation_reversed = subset.z.values[::-1, :]
ls = LightSource(azdeg=315, altdeg=45)
hillshade = ls.hillshade(elevation_reversed, vert_exag=2.5, dx=1, dy=1)

# Step 3: Plot the Hillshaded Relief with Cartopy
fig = plt.figure(figsize=(12, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Generate the normal map from the reversed elevation data
normal_map = generate_normal_map(elevation_reversed)

# Step 2: Plot Sparse Analysis
fig, ax = pvdeg.geospatial.plot_sparse_analysis(
    visual_demo_ds, data_var="cycles", method="linear", res=400j, ax=ax
)

# Step 3: Overlay the Normal Map
ax.imshow(
    normal_map,
    extent=[lon_min, lon_max, lat_min, lat_max],
    transform=ccrs.PlateCarree(),
    origin="upper",
)

# Step 4: Add Additional Features and Display the Plot
ax.add_feature(cfeature.COASTLINE, edgecolor="black", linewidth=0.5)
ax.add_feature(cfeature.BORDERS, edgecolor="black", linewidth=0.3)
ax.add_feature(cfeature.LAKES, edgecolor="blue", facecolor="none")
plt.title("Sparse Analysis with Normal Map Overlay", fontsize=16)

# Show the final plot
plt.show()

In [ ]:
from scipy.ndimage import zoom


# Step 1: Generate the Normal Map
def generate_normal_map(heightmap):
    grad_x, grad_y = np.gradient(heightmap)
    grad_z = np.ones_like(heightmap)
    length = np.sqrt(grad_x**2 + grad_y**2 + grad_z**2)
    nx = grad_x / length
    ny = grad_y / length
    nz = grad_z / length
    return np.stack((nx, ny, nz), axis=-1)


# Step 2: Simulate Lighting with the Normal Map
def apply_lighting(normal_map, light_dir):
    light_dir = light_dir / np.linalg.norm(light_dir)  # Normalize light direction
    intensity = np.dot(normal_map, light_dir)  # Compute dot product
    intensity = np.clip(intensity, 0, 1)  # Clamp to [0, 1]
    return intensity


# Generate the normal map and compute lighting
normal_map = generate_normal_map(elevation_reversed)
light_direction = np.array([1, 1, 1])  # Light source direction
lighting = apply_lighting(normal_map, light_direction)


fig = plt.figure(figsize=(12, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())


# Step 3: Plot Sparse Analysis and Apply Lighting
fig, ax = pvdeg.geospatial.plot_sparse_analysis(
    visual_demo_ds, data_var="cycles", method="linear", res=400j, ax=ax
)

# Extract the color data from the plot
analysis_image = ax.images[0]
color_data = analysis_image.get_array()
norm = Normalize(vmin=color_data.min(), vmax=color_data.max())

# Resample the lighting array to match the color data dimensions
lighting_resampled = zoom(
    lighting,
    (color_data.shape[0] / lighting.shape[0], color_data.shape[1] / lighting.shape[1]),
)

# Ensure dimensions match
assert (
    lighting_resampled.shape == color_data.shape
), "Lighting and color data shapes do not match!"

# Apply lighting to the colormap
shaded_colors = plt.cm.viridis(norm(color_data))
shaded_colors[..., :3] *= lighting_resampled[..., np.newaxis]

# Update the analysis plot with the shaded colors
analysis_image.set_data(shaded_colors)

# Optional: Add map features and display the plot
ax.add_feature(cfeature.COASTLINE, edgecolor="black", linewidth=0.5)
ax.add_feature(cfeature.BORDERS, edgecolor="black", linewidth=0.3)
ax.add_feature(cfeature.LAKES, edgecolor="blue", facecolor="none")
plt.title("Sparse Analysis with Normal Map Lighting", fontsize=16)

# Display the final plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

reversed = subset.z.values[::-1, :]

reversed[reversed < 0] = 0  # fill to zero

reversed = reversed**1 / 5

# Create the 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection="3d")

lat = subset.latitude.values  # [::-1].values  # Reverse latitude if needed
lon = subset.longitude.values

# Create a meshgrid of latitude and longitude
LAT, LON = np.meshgrid(lat, lon, indexing="ij")  # Use 'ij' to match (147, 345)

# `reversed` is your elevation or data matrix
# Now you can plot
surf = ax.plot_surface(LAT, LON, reversed, cmap="terrain", edgecolor="none")

In [ ]:
relief_gmt

In [ ]:
fig = pygmt.Figure()

fig.grdview(
    grid=relief_gmt.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)),
    # Sets the view azimuth as 130 degrees, and the view elevation as 30
    # degrees
    perspective=[130, 30],
    # Sets the x- and y-axis labels, and annotates the west, south, and east
    # axes
    frame=["xa", "ya", "WSnE"],
    # Sets a Mercator projection on a 15-centimeter figure
    projection="M15c",
    # Sets the height of the three-dimensional relief at 1.5 centimeters
    zsize="1.5c",
)
fig.show()